# downloading data form github  repo

In [1]:
from pydriller import Repository

repo_url = "spring-framework"

for commit in Repository(repo_url).traverse_commits():
    print(commit.__dict__)   
    break   


{'_c_object': <git.Commit "8119659fb1e4ae6fabe8897c42ba7629fda07b21">, '_conf': <pydriller.utils.conf.Conf object at 0x00000166CD9A6CE0>, '_stats_cache': None}


In [2]:
# to download repo wirte in the terminal git clone "github link"

In [3]:
import csv
'''
with open("fixed_file.csv", "w", newline="", encoding="utf-8") as f:
    df = csv.writer(f)
    df.writerow(["Hash", "Message", "Type", "Old_Code", "New_Code"])  

    for commit in Repository(repo_url).traverse_commits():
        if "fix" in commit.msg.lower():
            for m in commit.modified_files:
                if m.filename.endswith(".java"):
                    df.writerow([
                        commit.hash,
                        commit.msg,
                        m.change_type.name,
                        m.source_code_before, 
                        m.source_code 
                    ])
'''                    




'\nwith open("fixed_file.csv", "w", newline="", encoding="utf-8") as f:\n    df = csv.writer(f)\n    df.writerow(["Hash", "Message", "Type", "Old_Code", "New_Code"])  \n\n    for commit in Repository(repo_url).traverse_commits():\n        if "fix" in commit.msg.lower():\n            for m in commit.modified_files:\n                if m.filename.endswith(".java"):\n                    df.writerow([\n                        commit.hash,\n                        commit.msg,\n                        m.change_type.name,\n                        m.source_code_before, \n                        m.source_code \n                    ])\n'

### function to download commits from repo

In [4]:
import os
def extract_fixes(repo_path, output_file="fixed_file.csv"):
  
    file_exists = os.path.isfile(output_file)

    with open(output_file, "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        if not file_exists:
            writer.writerow(["Hash", "Message", "Type", "Old_Code", "New_Code"])
        
        for commit in Repository(repo_path).traverse_commits():
            if "fix" in commit.msg.lower():
                for m in commit.modified_files:
                    if m.filename.endswith(".java"): #just java files with fix commits
                        writer.writerow([
                            commit.hash,
                            commit.msg,
                            m.change_type.name,
                            m.source_code_before or "",
                            m.source_code or ""
                        ])



In [5]:
#extract_fixes("defects4j")

In [6]:
#extract_fixes("accumulo")

In [7]:
#extract_fixes("camel")

In [8]:
extract_fixes("ISSRE2023-BugSeverityPrediction/")

In [9]:
import pandas as pd  
df=pd.read_csv("fixed_file.csv")
df

,Hash,Message,Type,Old_Code,New_Code
0,7a55ce48a93f004c7acc0b51c04bb39be2c6b733,Handle CancellationException in JdkClientHttpR...,MODIFY,/*\n * Copyright 2002-present the original aut...,/*\n * Copyright 2002-present the original aut...
1,7a55ce48a93f004c7acc0b51c04bb39be2c6b733,Handle CancellationException in JdkClientHttpR...,ADD,NaN,package org.springframework.http.client;\n\nim...
2,ffc785471bbe579aaef282720baef00a44d46435,Fix checkstyle error,MODIFY,/*\n * Copyright 2002-present the original aut...,/*\n * Copyright 2002-present the original aut...
3,2b7f88ee449fb37c81f2e3649beaa643f9537357,Fix annotation arrays support in ClassFile met...,MODIFY,/*\n * Copyright 2002-present the original aut...,/*\n * Copyright 2002-present the original aut...
4,2b7f88ee449fb37c81f2e3649beaa643f9537357,Fix annotation arrays support in ClassFile met...,MODIFY,/*\n * Copyright 2002-present the original aut...,/*\n * Copyright 2002-present the original aut...
...,...,...,...,...,...
14740,375c508894d7cd6ac8012424cf12a4a18e401788,CAMEL-3768: Fixed issue with Jetty continuatio...,ADD,NaN,/**\n * Licensed to the Apache Software Founda...
14741,375c508894d7cd6ac8012424cf12a4a18e401788,CAMEL-3768: Fixed issue with Jetty continuatio...,ADD,NaN,/**\n * Licensed to the Apache Software Founda...
14742,521066816acbb28fca0442324d1d914d6d54fd31,CAMEL-3791，CAMLE-3792 Fix some issues of Error...,MODIFY,/**\n * Licensed to the Apache Software Founda...,/**\n * Licensed to the Apache Software Founda...
14743,521066816acbb28fca0442324d1d914d6d54fd31,CAMEL-3791，CAMLE-3792 Fix some issues of Error...,ADD,NaN,/**\n * Licensed to the Apache Software Founda...


In [10]:
import re
def remove_comments(code):
    if not isinstance(code, str): 
        return code
    code=re.sub(r'/\*.*?\*/', '', code, flags=re.DOTALL)  
    code=re.sub(r'//.*', '', code) 
    return code.strip()


In [11]:
df[["Old_Code", "New_Code"]] = df[["Old_Code", "New_Code"]].applymap(remove_comments)

C:\Users\hp\AppData\Local\Temp\ipykernel_5328\2932069089.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[["Old_Code", "New_Code"]] = df[["Old_Code", "New_Code"]].applymap(remove_comments)


In [13]:
print(df["New_Code"].iloc[0])

package org.springframework.http.client;

import java.io.FilterInputStream;
import java.io.IOException;
import java.io.InputStream;
import java.io.UncheckedIOException;
import java.net.URI;
import java.net.http.HttpClient;
import java.net.http.HttpRequest;
import java.net.http.HttpResponse;
import java.net.http.HttpTimeoutException;
import java.nio.ByteBuffer;
import java.time.Duration;
import java.util.Collections;
import java.util.Locale;
import java.util.Set;
import java.util.TreeSet;
import java.util.concurrent.CancellationException;
import java.util.concurrent.CompletableFuture;
import java.util.concurrent.ExecutionException;
import java.util.concurrent.Executor;
import java.util.concurrent.Flow;
import java.util.concurrent.TimeUnit;
import java.util.concurrent.atomic.AtomicBoolean;

import org.springframework.http.HttpHeaders;
import org.springframework.http.HttpMethod;
import org.springframework.lang.Nullable;
import org.springframework.util.StreamUtils;
import org.springframewo

In [12]:
print(df["Type"].unique())

['MODIFY' 'ADD' 'RENAME' 'DELETE']


In [14]:
a=['MODIFY', 'ADD', 'RENAME', 'DELETE']
for t in a:
    print(f"{t}: {len(df[df['Type'] == t])}")


MODIFY: 13226
ADD: 1011
RENAME: 328
DELETE: 180


In [15]:
df=df[df["Type"] == "MODIFY"]


In [16]:
df=df.reset_index(drop=True)
df

,Hash,Message,Type,Old_Code,New_Code
0,7a55ce48a93f004c7acc0b51c04bb39be2c6b733,Handle CancellationException in JdkClientHttpR...,MODIFY,package org.springframework.http.client;\n\nim...,package org.springframework.http.client;\n\nim...
1,ffc785471bbe579aaef282720baef00a44d46435,Fix checkstyle error,MODIFY,package org.springframework.http.client;\n\nim...,package org.springframework.http.client;\n\nim...
2,2b7f88ee449fb37c81f2e3649beaa643f9537357,Fix annotation arrays support in ClassFile met...,MODIFY,package org.springframework.core.type.classrea...,package org.springframework.core.type.classrea...
3,2b7f88ee449fb37c81f2e3649beaa643f9537357,Fix annotation arrays support in ClassFile met...,MODIFY,package org.springframework.core.type;\n\nimpo...,package org.springframework.core.type;\n\nimpo...
4,81b4020fc6392025e5e55b2d26b7cd5bac1ca7fe,Do not load concrete types in annotation metad...,MODIFY,package org.springframework.core.type.classrea...,package org.springframework.core.type.classrea...
...,...,...,...,...,...
13221,f37b27a8e4eccd58ec782a5e8e24abf5cb9c9ed9,CAMEL-3769: Fixed issue when using starttls wi...,MODIFY,package org.apache.camel.component.mail;\n\nim...,package org.apache.camel.component.mail;\n\nim...
13222,144f3aa0b0796d690c9a55d18b7816c1e9e44004,Fixed javadoc wrong for splitter DSL\n\ngit-sv...,MODIFY,package org.apache.camel.model;\n\nimport java...,package org.apache.camel.model;\n\nimport java...
13223,375c508894d7cd6ac8012424cf12a4a18e401788,CAMEL-3768: Fixed issue with Jetty continuatio...,MODIFY,package org.apache.camel.component.jetty;\n\ni...,package org.apache.camel.component.jetty;\n\ni...
13224,521066816acbb28fca0442324d1d914d6d54fd31,CAMEL-3791，CAMLE-3792 Fix some issues of Error...,MODIFY,package org.apache.camel.processor;\n\nimport ...,package org.apache.camel.processor;\n\nimport ...
